# GMM 

In [249]:
import numpy as np
import math
from scipy.stats import multivariate_normal
from random import choices
from numpy import linalg as LA


"""
This part is previous wrong initialization making each element in the weight Matrix the same and then 
"""
# def weight_initialize(data_array, k):
#     # data_array is N X M data array (Matrix)
#     # get the len of data array(i.e., N) and its dimension(i.e., M)
#     len_data = len(data_array)
#     dimen_data = len(data_array[0])
    
#     # created a weight matrix N X K and set each elements into 0
#     # source: https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html
#     weightMat = np.zeros([len_data, k])
    
#     # initialize the weight Matrix and for each data point, the weight[i][k]
#     # refer the probability of point i belonging into Gaussian distribution k
#     # set each into equal probability
#     for i in range(len_data): # for each data point
#         for j in range(k):
#             weightMat[i][j] = float(1.0)/float(k)
#             ## weightMat[i][k] = float(1.0)/float(k)
#     print(weightMat.shape)
#     print(weightMat[:5])
    
#     return weightMat


"""
This part is correct initialization, for each data point i, the element which refers to the index of the 
smallest distance between this data point and all the K Gaussians(likewise the K centriods in k-means algorithms), 
is set into 1, namely, the probablity of data point i belonging to a certain gaussian (the smallest distance) is
100% initially
"""

def weight_initialize(data_array, k):
    # data_array is N X M data array (Matrix)
    # get the len of data array(i.e., N) and its dimension(i.e., M)
    len_data = len(data_array)
    dimen_data = len(data_array[0])
    
    # created a weight matrix N X K and set each elements into 0
    # source: https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html
    weightMat = np.zeros([len_data, k])
    
    #randomly pick k data out of the array as the initial gaussians  
    #source: https://docs.python.org/3/library/random.html
    K_chosen = choices(range(len_data), k = k) ## a list
    gaussians_initial= data_array[K_chosen]
    
    print(gaussians_initial)
    
    for i in range(len_data):
        #define a list storing the distance of between each point and the gaussian's center
        distance = np.zeros(k)
        for j in range(k):
            
            ## get the 2-norm distance between the data point i and the gaussian center j
            ## https://docs.scipy.org/doc/numpy-1.9.3/reference/generated/numpy.linalg.norm.html
#             print(data_array[i, :] - gaussians_initial[j, :])
            tt = LA.norm(data_array[i, :] - gaussians_initial[j, :]) 
#             print(tt)
            distance[j] = tt
            #distance[j] = LA.norm(data_array[i, :] - gaussians_initial[j, :]) 
        ## return the index of the smallest distance, i.e., the closest gaussian
        # source : https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html    
        index_closest = np.argsort(distance)[0]
        weightMat[i][index_closest] = 1.0
        
    print(weightMat.shape)
    print(weightMat[:5])
        
    return weightMat
            

# When get weightMat from weight_initialize() function, for the first time we also
# need to get the initial mean and convariance by calling the M_step and then
# further we can enter the iteration loop
def M_step(data_array, weightMat, k):
    
    # data_array is N X M data array (Matrix)
    # get the len of data array(i.e., N) and its dimension(i.e., M)
    len_data = len(data_array)
    dimen_data = len(data_array[0])
#     print(k, dimen_data)
    
    ## initiate the mean matrix and the convariance matrix into 0
     ## meanMat is a K X M matrix; 
     ##  covMat is a list of M X M matrix (K Matrix elements)
    meanMat = np.zeros([k, dimen_data])
    
    covMat = []
    
    for i in range(k):
        covMat.append(np.zeros([dimen_data, dimen_data]))
    ## a question: why [covMat.append(np.zeros([dimen_data,dimen_data])) for i in range(k)] failed to work??
    
    """
    calculate the mean Matrix and convriance Matrix here
    """
    
    ## here is the tempt mean
    for j in range(len_data):
        for t in range(k):
            meanMat[t] += weightMat[j][t]*data_array[j] 
    # add the weight of each gaussian(i.e., cluster) together, here we need add elements of each colums together  
    ##https://docs.scipy.org/doc/numpy/reference/generated/numpy.sum.html
    weight_gaussian = np.sum(weightMat, axis=0) 
#     print("??")
#     print(weight_gaussian)
#     print("end")
    ## calculate the final mean Matrix
    for i in range(k):
        meanMat[i] = meanMat[i]/weight_gaussian[i]
    
    ## here is to get the tempt convirance
    for i in range(len_data):
        for j in range(k):
            for m in range(dimen_data):
                for n in range(dimen_data):
                    covMat[j][m][n] += weightMat[i][j]*(data_array[i][m] - meanMat[j][m])*(data_array[i][n] - meanMat[j][n])
    
    """ bad: invalid syntax
    ## here is to get the tempt convirance
    for i in range(len_data):
        for j in range(k):
            [covMat[j][m][n] += weightMat[i][j]*(data_array[i][m] - meanMat[j][m])*(data_array[i][n] - meanMat[j][n]) for m in range(dimen_data) for n in range(dimen_data)]
    """
    
    ## calculate the final covirance Matrix
    
    for i in range(k):
        ## covMat[i] = float(covMat[i])/float(weight_gaussian[i]) is wrong as only length-1 arrays can be converted to Python scalars
        covMat[i] = covMat[i]/weight_gaussian[i]
        
    
    return meanMat, covMat


## Here the E_step is the return the new Weight Matrix

def E_step(data_array, k, meanMat, covMat, weightMat_old):
    
    # data_array is N X M data array (Matrix)
    # get the len of data array(i.e., N) and its dimension(i.e., M)
    len_data = len(data_array)
    dimen_data = len(data_array[0])
    
    # add the weight of each gaussian(i.e., cluster) together, here we need add elements of each colums together  
    ##https://docs.scipy.org/doc/numpy/reference/generated/numpy.sum.html
    weight_gaussian = np.sum(weightMat_old, axis=0)
    
    ## before calculate the new weight Matrix we initialize it into 0
    weightMat_new = np.zeros([len_data, k])
    
    for i in range(len_data):
        for j in range(k):
            # get the probablity by using the multivariate normal pdf
            ## https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.multivariate_normal.html
            pro = multivariate_normal.pdf(data_array[i], meanMat[j], covMat[j])
            ## the tempt weightMat_new
            weightMat_new[i][j] = pro*weight_gaussian[j]
            
        ## here is the final weightMat_new
        weightMat_new[i] = weightMat_new[i]/np.sum(weightMat_new[i])
        
    return weightMat_new
    
    

def GMM(data_array, k, iteration):
    
    weightMat_old = weight_initialize(data_array, k)
    
    while iteration>1:
        iteration -=1
        
#         print(iteration)
#         print("The weightMat_old is :", weightMat_old)
        
        meanMat, covMat = M_step(data_array, weightMat_old, k)     
        
        
#         print(meanMat)
#         print("####")
#         print(covMat)
#         print("##")
        
        weightMat_new = E_step(data_array, k, meanMat, covMat, weightMat_old)
#         print(iteration)
#         print("The weightMat_new is :", weightMat_new)
        
        weightMat_old = weightMat_new
        
#         print(weightMat_old)
        
    return meanMat, covMat

    
if __name__ == "__main__":
    input_path1 = "/Users/deshenghu/Dropbox/Dataset_CS6220/2gaussian.txt"
    input_path2 = "/Users/deshenghu/Dropbox/Dataset_CS6220/3gaussian.txt"
    ## source: https://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html
    data1 = np.loadtxt(input_path1)
    data2 = np.loadtxt(input_path2)
    #print(data[-1:])
    meanMat1, covMat1 = GMM(data1, 2, 25)
    meanMat2, covMat2 = GMM(data2, 3, 80)
    print("This is for 2 Gaussian:")
    for i in range(2):
        print("Gaussian {}, mean matrix {} and covariance matrix {}".format(i, meanMat1[i], covMat1[i]))
        
    print("This is for 3 Gaussian:")
    for i in range(3):
        print("Gaussian {}, mean matrix {} and covariance matrix {}".format(i, meanMat2[i], covMat2[i]))
    

[[ 7.5842403   3.6109329 ]
 [ 8.25532046  5.12825222]]
(6000, 2)
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]
[[ 3.62795109  0.4059141 ]
 [ 6.67554898  1.9710692 ]
 [ 2.10181757  6.13741452]]
(6000, 3)
[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]]
This is for 2 Gaussian:
Gaussian 0, mean matrix [ 3.03043786  3.04882617] and covariance matrix [[ 1.07266452  0.02245989]
 [ 0.02245989  2.9103095 ]]
Gaussian 1, mean matrix [ 7.03092943  3.99329205] and covariance matrix [[ 0.9451349   0.48072411]
 [ 0.48072411  0.98661324]]
This is for 3 Gaussian:
Gaussian 0, mean matrix [ 3.11564032  2.38595739] and covariance matrix [[ 1.00352732  0.09619581]
 [ 0.09619581  2.28080539]]
Gaussian 1, mean matrix [ 7.0139545   3.98485373] and covariance matrix [[ 0.97332791  0.49427545]
 [ 0.49427545  0.99778653]]
Gaussian 2, mean matrix [ 2.8269699   3.98269921] and covariance matrix [[ 0.97736951  0.18618389]
 [ 0.18618389  2.37051071]]


In [112]:
input_path1 = "/Users/deshenghu/Dropbox/Dataset_CS6220/2gaussian.txt"
data = np.loadtxt(input_path1)
data.shape[0]

6000

In [233]:
import csv

##The “mnist_train_fashion.csv” is downloaded from webpage of this link : https://www.kaggle.com/zalando-research/fashionmnist
## here we only use the training data set for running the program

# data_array_fashion is to store the training data
data_array_fashion = []
with open("/Users/deshenghu/Dropbox/Dataset_CS6220/mnist_train_fashion.csv") as t:
    #read csv file using csv.reader()
    #source : https://docs.python.org/3/library/csv.html
    reader = csv.reader(t)
    for row in reader:
        data_array_fashion.append(row)
        

# get the lables of the training data, which is the first element of each row
data_fashion_lable = [row[0] for row in data_array_fashion]

# delete the lable which is the first element of each row
for i in range(len(data_array_fashion)):
    del data_array_fashion[i][0]
    
data_fashion_tempt =  np.array(data_array_fashion)
# because each element ("number") in the array is str and thus need to be converted
data_fashion_list = [list(map(int, data_fashion_tempt[j])) for j in range(len(data_fashion_tempt))]

data_fashion_training = np.matrix(data_fashion_list)


In [245]:
from sklearn.mixture import GaussianMixture as GMM
model = GMM(n_components = 10, covariance_type ="diag")
model.fit(data_fashion_training)
t= model.score(data_fashion_training)
print(t)


-365.016805832


In [246]:
from sklearn.metrics import completeness_score
prediction = model.predict(data_fashion_training)
completeness_score(data_fashion_lable, prediction)

0.30081468600940042

In [247]:
import csv

##The “mnist_train_fashion.csv” is downloaded from webpage of this link : https://www.kaggle.com/zalando-research/fashionmnist
## here we only use the training data set for running the program

# data_array_fashion is to store the training data
spam = []
with open("/Users/deshenghu/Dropbox/Dataset_CS6220/spambase/spambase.data") as t:
    for row in t:
        spam.append(list(map(float, row.replace("\n", "").split(","))))

spam = np.matrix(spam)

from sklearn.mixture import GaussianMixture as GMM
model = GMM(n_components = 2, covariance_type ="diag")
model.fit(spam)
t= model.score(spam)
print(t)




-7.42381427588
